In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
df=pd.read_csv("datanum1.csv")
l=len(df.index)
df=df.drop('Timestamp',axis = 1)
df=df.drop('Name', axis = 1)
df=df.drop('12th Stream',axis = 1)
sub=['CIVIL','CSE','ECE','EEE','MECH','BME','CHEM']


In [ ]:
data={'tenth':[88],'12th':[90],'creative':[3],'writing':[2],'outdoorwork':[1], 'future':[1], 'groupwork':[1],'enjoymost':[1],'enjoyleast':[1],'clubmost':[1],'clubleast':[1],'projectliked':[1],'projectdisliked':[1],'noengineering':[1],'drawing':[1],'futurejob':[2],'interest':[3],'Opted':[0],'satisfied':[0]}
df1=pd.DataFrame(data)
df1['index']=l;
l=l+1
df=df.append(df1,ignore_index=True)
df=df.fillna(0)
df.tail(3)


,tenth,12th,creative,writing,outdoorwork,future,groupwork,enjoymost,enjoyleast,clubmost,clubleast,projectliked,projectdisliked,noengineering,drawing,futurejob,interest,Opted,satisfied,index
1063,88.0,85.0,2,2,1,4,1,10,9,8,3,5,1,13,2,3,4,4,1,1063
1064,89.0,87.0,1,1,1,4,1,10,2,2,1,6,3,1,2,3,5,5,1,1064
1065,88.0,90.0,3,2,1,1,1,1,1,1,1,1,1,1,1,2,3,0,0,1065


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
	df, df['Opted'], test_size = 0.4)
print("Actual Output :",y_test)

Actual Output : 1052    3
749     2
529     4
139     1
124     3
       ..
598     5
312     2
612     2
147     1
655     1
Name: Opted, Length: 427, dtype: int64


In [ ]:
svc_model = SVC(C=0.1, kernel='poly', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False,tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, random_state=None)
svc_model.fit(X_train, y_train)
prediction = svc_model.predict(X_test)
print("Predicted Value:",prediction)

Predicted Value: [2 2 4 1 3 4 1 2 1 4 1 2 2 1 6 2 3 2 1 2 4 6 5 5 2 2 3 1 2 2 2 5 4 2 3 2 2
 2 1 5 1 1 2 1 2 3 2 2 1 2 2 1 2 2 2 4 2 4 2 2 1 5 2 4 2 2 1 3 2 4 2 1 2 5
 2 4 2 2 5 2 3 3 2 4 3 3 4 2 2 1 4 4 2 1 2 5 4 3 2 1 5 4 1 1 5 2 5 2 1 2 5
 1 2 3 2 4 1 4 3 2 2 2 3 5 4 2 5 4 3 3 4 5 1 4 2 4 5 4 1 4 2 1 2 2 3 1 4 3
 2 2 2 1 4 2 1 3 2 2 2 3 4 5 2 5 1 2 2 1 6 1 1 2 5 1 1 2 1 4 1 0 2 6 1 3 4
 2 6 4 1 2 1 2 4 1 2 5 1 2 2 4 5 2 4 2 1 3 2 5 2 5 4 3 5 2 1 3 5 4 3 1 4 4
 1 4 0 1 1 2 5 1 2 2 2 2 4 5 5 2 4 2 2 2 2 4 4 1 5 2 2 5 1 6 1 1 1 2 2 1 3
 3 4 1 1 2 3 2 2 5 3 2 5 1 2 2 1 2 2 5 3 4 2 4 5 1 1 1 1 4 1 2 1 1 4 4 1 4
 2 1 4 3 2 1 3 1 5 5 3 5 4 2 3 2 2 5 2 1 1 5 5 1 2 5 3 1 2 4 2 2 5 2 1 2 2
 2 1 5 2 4 5 2 1 4 6 2 2 2 2 5 2 1 2 1 5 2 2 5 3 3 3 3 3 2 4 2 2 4 4 6 3 2
 5 4 1 4 4 4 2 4 4 5 4 3 1 1 5 5 2 1 4 1 3 2 2 1 2 2 6 2 5 3 1 3 3 2 4 5 5
 5 1 2 4 4 1 5 1 1 3 3 5 1 2 5 5 2 2 1 1]


In [ ]:
acc = metrics.accuracy_score(y_test, prediction)
p = svc_model.predict(df1)
print("Recommended Course: ",p+1)
print("Accuracy:",acc)

Recommended Course:  [1]
Accuracy: 0.8852459016393442


In [ ]:
# Save machine learning model
import pickle
filename = 'finalized_model.sav'
pickle.dump(svc_model, open(filename, 'wb'))

In [ ]:
print(metrics.mean_squared_error(y_test, prediction))

0.28337236533957844
